## pyPortfolioOpt Mean Variance Optimization
### Out-of-sample result is very bad

In [2]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import pprint
import inspect  # <--- ADD THIS LINE
from IPython.display import display, Markdown

# --- 1. PANDAS & IPYTHON OPTIONS ---
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 3000)
pd.set_option('display.float_format', '{:.6f}'.format)
%load_ext autoreload
%autoreload 2

# --- 2. PROJECT PATH CONFIGURATION ---
NOTEBOOK_DIR = Path.cwd()
PARENT_DIR = NOTEBOOK_DIR
ROOT_DIR = NOTEBOOK_DIR.parent  # Adjust if your notebook is in a 'notebooks' subdirectory
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'

# Add 'src' to the Python path to import custom modules
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- 3. IMPORT CUSTOM MODULES ---
import utils

# --- 4. INITIAL_CAPITAL ---
INITIAL_CAPITAL = 100000

# --- 5. RISK FREE ANNUAL RATE ---
RISK_FREE_ANNUAL_RATE = 0.04

# --- 6. VERIFICATION ---
print("--- Path Configuration ---")
print(f"✅ Project Root: {ROOT_DIR}")
print(f"✅ Parent Dir:   {PARENT_DIR}")
print(f"✅ Notebook Dir: {NOTEBOOK_DIR}")
print(f"✅ Data Dir:     {DATA_DIR}")
print(f"✅ Source Dir:   {SRC_DIR}")
assert all([ROOT_DIR.exists(), DATA_DIR.exists(), SRC_DIR.exists()]), "A key directory was not found!"

print("\n--- Module Verification ---")
print(f"✅ Successfully imported 'utils' and 'plotting_utils'.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
--- Path Configuration ---
✅ Project Root: c:\Users\ping\Files_win10\python\py311\stocks
✅ Parent Dir:   c:\Users\ping\Files_win10\python\py311\stocks\notebooks_PyPortfOpt
✅ Notebook Dir: c:\Users\ping\Files_win10\python\py311\stocks\notebooks_PyPortfOpt
✅ Data Dir:     c:\Users\ping\Files_win10\python\py311\stocks\data
✅ Source Dir:   c:\Users\ping\Files_win10\python\py311\stocks\src

--- Module Verification ---
✅ Successfully imported 'utils' and 'plotting_utils'.


In [3]:
df = pd.read_parquet(DATA_DIR / 'df_adj_close.parquet')
print(f'df:\n{df}')

df:
Ticker              A        AA       AAL      AAON       AAPL       ABBV     ABEV       ABNB        ABT      ACGL      ACHR       ACI        ACM        ACN       ACWI      ACWX       ADBE       ADC        ADI       ADM        ADP       ADSK      ADT        AEE      AEG        AEM        AEP        AER       AES        AFG        AFL      AFRM       AGCO       AGG       AGI     AGNC       AIG       AIQ      AIRR        AIT        AIZ        AJG      AKAM        AL        ALB       ALC      ALGM       ALGN       ALK        ALL       ALLE      ALLY       ALNY      ALSN        ALV        AM       AMAT      AMCR        AMD        AME       AMGN       AMH      AMLP        AMP        AMT       AMX       AMZN         AN       ANET        AON       AOS       APA        APD       APG        APH        APO        APP       APPF      APTV        AR      ARCC        ARE       ARES       ARGX      ARKK      ARMK        ARW       ASML       ASND        ASR      ASTS       ASX       ATI        AT

### Used 2023-2024 data for training. MVO's performance from Jan. to Feb. 2025 is bad. So MVO out-of-sample is useless. 

In [4]:
import pandas as pd
import numpy as np

# 2. --- This is the code to split your DataFrame ---

# Create a boolean mask for the years 2023 and 2024
mask = df.index.year.isin([2023, 2024])

# Apply the mask to get the first DataFrame
df_train = df[mask]

# Apply the inverse of the mask (using ~) to get the second DataFrame
df_remain = df[~mask]

test_mask = (df_remain.index.year == 2025) & df_remain.index.month.isin([1, 2])
df_test = df_remain.loc[test_mask]

In [5]:
import numpy as np
import pandas as pd
from pypfopt import risk_models, expected_returns
from pypfopt.efficient_frontier import EfficientFrontier
import sys

# ===================================================================
# PART 1: DEFINE ALL TOOLS AND FUNCTIONS
# ===================================================================

def prune_by_correlation(mu, S, weight_threshold=0.85, verbose=False):
    """Iteratively prunes a universe by removing the lower-return asset from any
    pair with correlation > weight_threshold, until no such pairs remain."""
    print(f"Pruning asset universe with correlation > {weight_threshold:.0%}...")
    mu_pruned, S_pruned = mu.copy(), S.copy()
    
    while True:
        corr_matrix = risk_models.cov_to_corr(S_pruned)
        corr_unstacked = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).stack()
        if corr_unstacked.empty or corr_unstacked.max() < weight_threshold: break
        
        ticker1, ticker2 = corr_unstacked.idxmax()
        ticker_to_drop = ticker1 if mu_pruned[ticker1] < mu_pruned[ticker2] else ticker2
        if verbose:
            print(f"  - High correlation: ({ticker1}, {ticker2}) = {corr_unstacked.max():.2f}. Dropping '{ticker_to_drop}'.")
        mu_pruned = mu_pruned.drop(ticker_to_drop)
        S_pruned = S_pruned.drop(index=ticker_to_drop, columns=ticker_to_drop)
        
    print(f"Finished pruning. {len(mu_pruned)} tickers remain.")
    return mu_pruned, S_pruned

def filter_by_returns(mu, S, quantile_to_keep=0.50):
    """Filters a universe by keeping only the top performers by expected return."""
    print(f"\nFiltering for top {quantile_to_keep:.0%} of assets by expected return...")
    return_cutoff = mu.quantile(1 - quantile_to_keep)
    tickers_to_keep = mu[mu >= return_cutoff].index
    mu_filtered, S_filtered = mu.loc[tickers_to_keep], S.loc[tickers_to_keep, tickers_to_keep]
    print(f"Finished filtering. {len(mu_filtered)} tickers remain.")
    return mu_filtered, S_filtered

def verify_correlation(S, weight_threshold):
    """Checks if any asset pairs have a correlation higher than the weight_threshold."""
    print(f"\nVerifying final asset list for correlations > {weight_threshold:.1%}...")
    sys.stdout.flush()
    corr_matrix = risk_models.cov_to_corr(S)
    corr_unstacked = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).stack()
    if corr_unstacked.empty or corr_unstacked.max() < weight_threshold:
        print(f"✅ VERIFICATION SUCCESS: No remaining pairs are above the {weight_threshold:.1%} correlation weight_threshold.")
    else:
        max_corr_pair = corr_unstacked.idxmax()
        print(f"❌ VERIFICATION FAILED: Highest correlation is {corr_unstacked.max():.2f} > {weight_threshold:.1%}.")
    sys.stdout.flush()

def verify_returns(final_mu, reference_mu, quantile_to_keep):
    """Checks if all assets in the final list meet the return quantile criteria."""
    print(f"\nVerifying final asset list against the top {quantile_to_keep:.1%} return quantile...")
    sys.stdout.flush()
    return_cutoff = reference_mu.quantile(1 - quantile_to_keep)
    min_return_in_final_list = final_mu.min()
    if min_return_in_final_list >= return_cutoff:
        print(f"✅ VERIFICATION SUCCESS: All assets meet the return cutoff for the top {quantile_to_keep:.1%}.")
        print(f"   (Min return in list: {min_return_in_final_list:.2%}, Cutoff was: {return_cutoff:.2%})")
    else:
        print(f"❌ VERIFICATION FAILED: At least one asset is below the return cutoff ({return_cutoff:.2%}).")
    sys.stdout.flush()

try:
    # ===================================================================
    # PART 2: PRE-OPTIMIZATION FILTERING & PRIMARY PORTFOLIO GENERATION
    # ===================================================================
    
    # --- Initial Data and Parameter Setup ---
    # Assume 'df_train' is your full DataFrame of historical prices for ALL candidate tickers.
    # df_train = pd.read_csv("tests/resources/stock_prices.csv", parse_dates=True, index_col="date")
    
    print(f"Starting with an initial universe of {len(df_train.columns)} tickers.")
    
    full_mu = expected_returns.mean_historical_return(df_train)
    full_S = risk_models.risk_matrix(df_train, method='ledoit_wolf')
    
    CORRELATION_weight_threshold = 0.80
    RETURN_QUANTILE_TO_KEEP = 0.10
    
    mu_after_pruning, S_after_pruning = prune_by_correlation(full_mu, full_S, weight_threshold=CORRELATION_weight_threshold, verbose=False)
    mu_final, S_final = filter_by_returns(mu_after_pruning, S_after_pruning, quantile_to_keep=RETURN_QUANTILE_TO_KEEP)
    
    print("\n" + "="*20 + " VERIFICATION " + "="*20)
    verify_correlation(S_final, CORRELATION_weight_threshold)
    verify_returns(mu_final, mu_after_pruning, RETURN_QUANTILE_TO_KEEP)
    print("="*54)

    # --- Run the Main Optimization ---
    print(f"\nRunning main optimizer on the final curated universe of {len(mu_final)} tickers...")
    ef = EfficientFrontier(mu_final, S_final, weight_bounds=(0, 0.5), solver='SCS')
    ef.risk_free_rate = RISK_FREE_ANNUAL_RATE
    
    # ===================== THE FIX IS HERE =====================
    # 1. Run the optimization to actually compute the weights
    raw_weights = ef.max_sharpe() 
    
    # 2. NOW that weights have been computed, we can clean them.
    main_weights = ef.clean_weights()
    # ===========================================================
    
    print("SUCCESS! Main optimization complete.")
    
    print("\n--- Primary Optimized Portfolio (from curated universe) ---")
    ef.portfolio_performance(verbose=True)
    print(f"Number of tickers with non-zero weight: {len(main_weights)}")

    # ===================================================================
    # PART 3: POST-OPTIMIZATION SENSITIVITY ANALYSIS
    # ===================================================================

    # Initialize storage for all portfolios (now including metrics)
    all_portfolios = {}  # Will store {'weights': ..., 'return': ..., 'volatility': ..., 'sharpe': ...}
    summary_data = []

    # Define weight threshold range
    weight_threshold_range = np.arange(0, 0.051, 0.005)

    for weight_threshold in weight_threshold_range:
        rounded_weight_threshold = round(weight_threshold, 5)
        
        # Filter tickers based on minimum weight threshold
        filtered_weights = {
            ticker: weight 
            for ticker, weight in main_weights.items() 
            if weight >= weight_threshold
        }
        
        # Skip if no tickers meet the threshold
        if not filtered_weights:
            continue
        
        # Rebalance weights to sum to 1.0
        total_filtered_weight = sum(filtered_weights.values())
        rebalanced_weights = {
            ticker: weight / total_filtered_weight 
            for ticker, weight in filtered_weights.items()
        }
        
        # Convert to Series for metric calculation
        rebalanced_weights_series = pd.Series(rebalanced_weights).reindex(mu_final.index).fillna(0)
        
        # Portfolio metrics
        p_return = np.dot(rebalanced_weights_series, mu_final)
        p_volatility = np.sqrt(np.dot(rebalanced_weights_series.T, np.dot(S_final, rebalanced_weights_series)))
        p_sharpe = (p_return - RISK_FREE_ANNUAL_RATE) / p_volatility

        # ✅ Save full portfolio data: weights + metrics
        all_portfolios[rounded_weight_threshold] = {
            'weights': rebalanced_weights,
            'return': p_return,
            'volatility': p_volatility,
            'sharpe': p_sharpe
        }

        # Store summary (for printing)
        summary_data.append({
            "Min Weight": f"{rounded_weight_threshold:.3f}",
            "Num Tickers": len(rebalanced_weights),
            "Return": f"{p_return:.2%}",
            "Volatility": f"{p_volatility:.2%}",
            "Sharpe Ratio": f"{p_sharpe:.2f}"
        })

    # Create and display summary DataFrame
    summary_df_train = pd.DataFrame(summary_data)
    print(summary_df_train.to_string(index=False))
    print("="*50)

except Exception as e:
    print(f"\n\nFATAL ERROR during processing: {e}")

Starting with an initial universe of 1518 tickers.
Pruning asset universe with correlation > 80%...
Finished pruning. 1460 tickers remain.

Filtering for top 10% of assets by expected return...
Finished filtering. 146 tickers remain.

==================== VERIFICATION ====================

Verifying final asset list for correlations > 80.0%...
✅ VERIFICATION SUCCESS: No remaining pairs are above the 80.0% correlation weight_threshold.

Verifying final asset list against the top 10.0% return quantile...
✅ VERIFICATION SUCCESS: All assets meet the return cutoff for the top 10.0%.
   (Min return in list: 48.85%, Cutoff was: 48.66%)

Running main optimizer on the final curated universe of 146 tickers...
SUCCESS! Main optimization complete.

--- Primary Optimized Portfolio (from curated universe) ---
Expected annual return: 252.0%
Annual volatility: 28.6%
Sharpe Ratio: 8.82
Number of tickers with non-zero weight: 146
Min Weight  Num Tickers  Return Volatility Sharpe Ratio
     0.000        

c:\Users\ping\Files_win10\python\py311\.venv\Lib\site-packages\scs\__init__.py:96: UserWarning: Converting A to a CSC (compressed sparse column) matrix; may take a while.
  warn(
c:\Users\ping\Files_win10\python\py311\.venv\Lib\site-packages\scs\__init__.py:126: UserWarning: Converting P to a CSC (compressed sparse column) matrix; may take a while.
  warn(


In [6]:
def display_portfolio_for_weight_threshold(portfolios_dict, target_weight_threshold):
    """
    Retrieves and prints the tickers, weights, and performance metrics 
    for a specific weight threshold from the stored portfolio data.

    Args:
        portfolios_dict (dict): Dictionary where key = threshold (float),
                                value = dict containing 'weights', 'return', etc.
        target_weight_threshold (float): Threshold to look up (e.g., 0.01 for 1.0%).
    """
    rounded_target = round(target_weight_threshold, 5)
    
    print(f"\n--- Recalling Portfolio for Weight Threshold: {rounded_target:.3f} ({rounded_target:.1%}) ---")

    if rounded_target not in portfolios_dict:
        print(f"ERROR: No data found for weight threshold {rounded_target:.3f}.")
        available_keys = [f"{k:.3f}" for k in sorted(portfolios_dict.keys())]
        print(f"Available thresholds: {', '.join(available_keys)}")
        return

    # Retrieve data
    record = portfolios_dict[rounded_target]
    weights = record['weights']
    p_return = record['return']
    p_volatility = record['volatility']
    p_sharpe = record['sharpe']

    # Print metrics
    print("--- Portfolio Summary ---")
    print(f"{'Number of Tickers':<20}: {len(weights):8}")
    print(f"{'Portfolio Return':<20}: {p_return:8.2%}")
    print(f"{'Portfolio Volatility':<20}: {p_volatility:8.2%}")
    print(f"{'Sharpe Ratio':<20}: {p_sharpe:8.2f}")

    # Print allocations
    sorted_weights = sorted(weights.items(), key=lambda x: x[1], reverse=True)
    print("Asset Allocation:")
    for ticker, weight in sorted_weights:
        weight_str = f"{weight:.2%}"
        print(f"  - {ticker:<6} : {weight_str:>7}")        

In [7]:
# Define the threshold you want to use
target_threshold = 0.005
rounded_threshold = round(target_threshold, 5)

if 'all_portfolios' in locals():
    if rounded_threshold in all_portfolios:
        # Step 1: Retrieve the rebalanced weights dict
        weight_dict = all_portfolios[rounded_threshold]['weights']  # {'APP': 0.21, 'COKE': 0.10, ...}
        
        # Step 2: Extract weights in the same order as df_train.columns (critical!)
        mvo_weights = np.array([weight_dict.get(ticker, 0.0) for ticker in df_train.columns])
        
        print(f"\nUsing MVO weights from threshold {rounded_threshold:.1%}:")
        # Pretty print the allocation using your display function
        display_portfolio_for_weight_threshold(all_portfolios, target_threshold)
    else:
        print(f"ERROR: No portfolio found for threshold {target_threshold:.3f}")
        available = sorted(all_portfolios.keys())
        print(f"Available thresholds: {[f'{t:.3f}' for t in available]}")
        mvo_weights = None
else:
    print("ERROR: 'all_portfolios' not found in memory.")
    mvo_weights = None

# Only proceed if we have valid weights
if mvo_weights is not None and len(mvo_weights) == len(df_train.columns):
    # Step 3: Get last price from training period
    LastPrice = np.array([1 / p for p in df_train.tail(1).to_numpy()[0]])  # Shares = weight / price

    # Step 4: Compute initial number of shares to buy
    Initial_Portfolio = np.multiply(mvo_weights, LastPrice) * INITIAL_CAPITAL    
    
    print(f"\nInitial Portfolio (number of shares):")
    for ticker, shares in zip(df_train.columns, Initial_Portfolio):
        if shares > 0:
            print(f"  - {ticker:<6} : {shares:>10.4f}")

    # Step 5: Simulate portfolio value over test period
    Portfolio_Assets = df_test.values @ Initial_Portfolio  # Matrix multiply: (time, tickers) x (tickers,) -> (time,)
    
    # Step 6: Create result DataFrame
    MVO_result = pd.DataFrame(Portfolio_Assets, index=df_test.index, columns=["Mean Var"])
    print(f'\nMVO_result (Portfolio Value Over Time):\n{MVO_result}')
else:
    print("Cannot compute portfolio simulation due to missing or mismatched weights.")


Using MVO weights from threshold 0.5%:

--- Recalling Portfolio for Weight Threshold: 0.005 (0.5%) ---
--- Portfolio Summary ---
Number of Tickers   :       12
Portfolio Return    :  252.24%
Portfolio Volatility:   28.60%
Sharpe Ratio        :     8.68
Asset Allocation:
  - APP    :  20.44%
  - FTAI   :  16.70%
  - SFM    :  12.72%
  - COKE   :  10.02%
  - GBTC   :   8.66%
  - GGAL   :   7.44%
  - CVNA   :   7.23%
  - SPOT   :   6.15%
  - NVDL   :   5.15%
  - BRBR   :   3.96%
  - PBR-A  :   0.85%
  - EAT    :   0.67%

Initial Portfolio (number of shares):
  - APP    :    63.1225
  - BRBR   :    52.5621
  - COKE   :    82.7462
  - CVNA   :    35.5438
  - EAT    :     5.0711
  - FTAI   :   116.5903
  - GBTC   :   116.9683
  - GGAL   :   119.9770
  - NVDL   :    77.6253
  - PBR-A  :    71.4590
  - SFM    :   100.1272
  - SPOT   :    13.7575

MVO_result (Portfolio Value Over Time):
                Mean Var
Date                    
2025-01-02 103392.690426
2025-01-03 105363.792407
2025-01-

In [8]:
def summarize_portfolio_performance_extended(
    portfolio_values,
    weight_threshold,
    portfolios_data,
    risk_free_rate_annual=RISK_FREE_ANNUAL_RATE,
    trading_days=252
):
    """
    Extended version with context: includes in-sample metrics and allocation info.
    """
    # === 1. Retrieve in-sample (training) metrics from all_portfolios ===
    rounded_threshold = round(weight_threshold, 5)
    
    if rounded_threshold not in portfolios_data:
        print(f"Warning: No training data found for threshold {rounded_threshold:.3f}")
        in_sample = None
    else:
        record = portfolios_data[rounded_threshold]
        in_sample = {
            'return': record['return'],
            'volatility': record['volatility'],
            'sharpe': record['sharpe'],
            'num_tickers': len(record['weights'])
        }

    # === 2. Compute out-of-sample (test) performance ===
    vals = np.array(portfolio_values)
    idx = portfolio_values.index if isinstance(portfolio_values, pd.Series) else range(len(vals))
    
    total_return = (vals[-1] - vals[0]) / vals[0]
    
    if isinstance(idx, pd.DatetimeIndex):
        time_in_years = (idx[-1] - idx[0]).days / 365.25
    else:
        time_in_years = len(vals) / trading_days

    cagr = (vals[-1] / vals[0]) ** (1 / time_in_years) - 1
    
    returns = pd.Series(vals).pct_change().dropna()
    annual_volatility = returns.std() * np.sqrt(trading_days)
    
    excess_return = cagr - RISK_FREE_ANNUAL_RATE
    sharpe_ratio = excess_return / annual_volatility if annual_volatility != 0 else 0.0
    
    cumulative_max = np.maximum.accumulate(vals)
    drawdowns = (vals - cumulative_max) / cumulative_max
    max_drawdown = abs(drawdowns.min())
    
    calmar_ratio = cagr / max_drawdown if max_drawdown != 0 else 0.0

    # === 3. Print Extended Summary ===
    print("\n" + "="*65)
    print(f"   PORTFOLIO PERFORMANCE REVIEW: THRESHOLD {rounded_threshold:.3f} ({rounded_threshold:.1%})")
    print("="*65)

    if in_sample:
        print(f"Training Period (In-Sample) Metrics:")
        print(f"  - Expected Return:     {in_sample['return']:.2%}")
        print(f"  - Expected Volatility: {in_sample['volatility']:.2%}")
        print(f"  - Expected Sharpe:     {in_sample['sharpe']:.2f}")
        print(f"  - Number of Tickers:   {in_sample['num_tickers']}")
    else:
        print("Training metrics: Not available")

    print("\nBacktest Period (Out-of-Sample) Performance:")
    print(f"  - Period:              {time_in_years:.2f} years")
    print(f"  - Initial Value:       ${vals[0]:,.2f}")
    print(f"  - Final Value:         ${vals[-1]:,.2f}")
    print(f"  - Total Return:        {total_return:.2%}")
    print(f"  - CAGR:                {cagr:.2%}")
    print(f"  - Annual Volatility:   {annual_volatility:.2%}")
    print(f"  - Sharpe Ratio:        {sharpe_ratio:.2f}")
    print(f"  - Max Drawdown:        {max_drawdown:.2%}")
    print(f"  - Calmar Ratio:        {calmar_ratio:.2f}")

    # === 4. Comparison: In-Sample vs Out-of-Sample ===
    if in_sample:
        print("\n" + "-"*40)
        print("  PERFORMANCE COMPARISON")
        print("-"*40)
        print(f"Return       : {in_sample['return']:.2%} (in) → {cagr:.2%} (out)")
        print(f"Volatility   : {in_sample['volatility']:.2%} (in) → {annual_volatility:.2%} (out)")
        print(f"Sharpe       : {in_sample['sharpe']:.2f} (in) → {sharpe_ratio:.2f} (out)")

    print("="*65)

# --- Usage ---
if 'MVO_result' in locals() and 'all_portfolios' in locals():
    summarize_portfolio_performance_extended(
        portfolio_values = MVO_result["Mean Var"],
        weight_threshold = 0.005,           # ← Change this to match your threshold
        portfolios_data = all_portfolios,
        risk_free_rate_annual = RISK_FREE_ANNUAL_RATE,
    )
else:
    print("ERROR: MVO_result or all_portfolios not found.")


   PORTFOLIO PERFORMANCE REVIEW: THRESHOLD 0.005 (0.5%)
Training Period (In-Sample) Metrics:
  - Expected Return:     252.24%
  - Expected Volatility: 28.60%
  - Expected Sharpe:     8.68
  - Number of Tickers:   12

Backtest Period (Out-of-Sample) Performance:
  - Period:              0.16 years
  - Initial Value:       $103,392.69
  - Final Value:         $102,660.39
  - Total Return:        -0.71%
  - CAGR:                -4.45%
  - Annual Volatility:   38.61%
  - Sharpe Ratio:        -0.22
  - Max Drawdown:        16.73%
  - Calmar Ratio:        -0.27

----------------------------------------
  PERFORMANCE COMPARISON
----------------------------------------
Return       : 252.24% (in) → -4.45% (out)
Volatility   : 28.60% (in) → 38.61% (out)
Sharpe       : 8.68 (in) → -0.22 (out)
